In [45]:
from datetime import datetime, timedelta
from backtesting import Backtest, Strategy
import pandas as pd
import ta
import numpy as np
import ta
from binance.client import Client
import ccxt
import mplfinance as mpf
import numpy as np
from math import floor
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import config

client = Client(config.api_key, config.api_secret)
# Create Binance client instance
# exchange = ccxt.binance({
#     'apiKey': config.api_key,
#     'secret': config.api_secret,
# })

In [46]:
def get_candle_1m_after(time):
    start = str(time)
    end = str(time + timedelta(hours=5))
    df = pd.DataFrame(client.get_historical_klines(symbol="BNBUSDT", interval='1m', start_str=start, end_str=end))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

def get_candle_1h_before(time):
    start = str(time - timedelta(hours=5))
    end = str(time)
    df = pd.DataFrame(client.get_historical_klines(symbol="BNBUSDT", interval='1h', start_str=start, end_str=end))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.set_index('time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [63]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

In [136]:
def get_data(symbol, interval='1h'):
    df = pd.DataFrame(client.get_historical_klines(symbol=symbol, interval=interval))
    df = df.iloc[:, :5]
    df.columns = ['time', 'open', 'high', 'low', 'close']
    df = df.astype(float)
    df.time = pd.to_datetime(df.time, unit='ms')
    mac = get_macd(df['close'], 26, 12, 9)
    ret = pd.concat([df, mac], join = 'inner', axis = 1)
    return ret

In [111]:
def get_signal(data):
    buy_price = []
    macd_signal = []

    for i in range(len(data)):
        try:
            if data['macd'][i-1] < data['signal'][i-1]:
                if data['macd'][i] > data['signal'][i]:
                    buy_price.append(data.iloc[i].close)
                    macd_signal.append(1)
                else:
                    buy_price.append(np.nan)
                    macd_signal.append(0)
            else:
                buy_price.append(np.nan)
                macd_signal.append(0)
        except:
                buy_price.append(np.nan)
                macd_signal.append(0)
    return pd.DataFrame({"buy_price": buy_price, "macd_signal": macd_signal})

In [131]:
def implement_macd_strategy(prices, data):
    buy_price = []
    sell_price = []
    macd_signal = []
    signal = 0

    for i in range(len(data)):
        if data['macd'][i] > data['signal'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif data['macd'][i] < data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)

    return pd.DataFrame( {'buy_price': buy_price, 'sell_price':sell_price, 'macd_signal':macd_signal} )

# buy_price, sell_price, macd_signal = implement_macd_strategy(googl['close'], googl_macd)

In [137]:
df = get_data('BNBUSDT')
# signal = get_signal(df)
# signal[signal['macd_signal'] == 1]
# sig = implement_macd_strategy(df['close'], df)
# sig

In [138]:
df

,time,open,high,low,close,macd,signal,hist
0,2023-03-21 15:00:00,335.2,337.9,335.0,337.7,0.000000,0.000000,0.000000
1,2023-03-21 16:00:00,337.8,339.9,336.9,337.5,-0.015954,-0.003191,-0.012764
2,2023-03-21 17:00:00,337.5,338.6,336.1,336.4,-0.116022,-0.025757,-0.090265
3,2023-03-21 18:00:00,336.4,337.5,335.0,337.2,-0.129282,-0.046462,-0.082820
4,2023-03-21 19:00:00,337.1,338.5,336.8,337.9,-0.082358,-0.053641,-0.028717
...,...,...,...,...,...,...,...,...
995,2023-05-02 03:00:00,326.4,326.4,325.2,325.8,-1.364031,-0.734339,-0.629691
996,2023-05-02 04:00:00,325.7,325.8,323.8,324.1,-1.587550,-0.904982,-0.682568
997,2023-05-02 05:00:00,324.0,324.6,322.9,324.6,-1.704694,-1.064924,-0.639770
998,2023-05-02 06:00:00,324.5,324.8,323.5,324.0,-1.824910,-1.216921,-0.607989


In [206]:
for x in range(len(df)):
    if signal.iloc[x].macd_signal == 1:
        before = get_candle_1h_before(df.iloc[x].name)
        print(before)
        break

                      Open   High    Low  Close
Time                                           
2023-04-29 19:00:00  322.0  323.1  321.9  322.9
2023-04-29 20:00:00  322.9  322.9  321.9  322.2
2023-04-29 21:00:00  322.3  322.5  321.7  322.0
2023-04-29 22:00:00  322.0  322.6  321.7  322.3
2023-04-29 23:00:00  322.3  322.4  321.4  321.9
2023-04-30 00:00:00  321.9  321.9  320.8  321.0


In [119]:
result = pd.concat([df, macd, signal], axis=1)
result = result[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'macd', 'signal', 'hist', 'buy_price', 'sell_price', 'macd_signal']]
result = result.iloc[:, 1:]
result.timestamp = pd.to_datetime(result.timestamp)
result.iloc[:10 : -1]

,timestamp,open,high,low,close,volume,macd,signal,hist,buy_price,sell_price,macd_signal
999,2023-04-30 11:00:00,322.2,322.4,321.6,321.7,3014.644,-0.337669,-0.463755,0.126086,NaN,NaN,0
998,2023-04-30 10:00:00,322.1,322.6,321.7,322.2,6509.091,-0.343461,-0.495277,0.151816,NaN,NaN,0
997,2023-04-30 09:00:00,322.1,322.3,321.7,322.1,4611.320,-0.396768,-0.533230,0.136463,NaN,NaN,0
996,2023-04-30 08:00:00,321.5,322.4,321.5,322.0,4693.970,-0.448853,-0.567346,0.118493,NaN,NaN,0
995,2023-04-30 07:00:00,322.1,322.4,321.5,321.6,5910.864,-0.498623,-0.596969,0.098347,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15,2023-03-20 10:00:00,337.3,338.7,336.5,337.3,19655.626,-0.798160,-1.019320,0.221160,NaN,NaN,0
14,2023-03-20 09:00:00,337.6,338.2,335.9,337.4,20436.360,-0.961057,-1.074610,0.113553,337.4,NaN,1
13,2023-03-20 08:00:00,334.7,338.2,334.2,337.6,39552.105,-1.165175,-1.102998,-0.062177,NaN,NaN,0
12,2023-03-20 07:00:00,333.5,335.8,333.5,334.7,42475.501,-1.429120,-1.087454,-0.341666,NaN,NaN,0


In [120]:
signals = result[result['macd_signal'] == 1]
signals.iloc[: :-1]

,timestamp,open,high,low,close,volume,macd,signal,hist,buy_price,sell_price,macd_signal
993,2023-04-30 05:00:00,321.9,322.3,321.7,322.3,4087.009,-0.577437,-0.648403,0.070966,322.3,NaN,1
957,2023-04-28 17:00:00,322.6,322.7,321.8,322.3,9173.150,-2.582274,-2.626530,0.044256,322.3,NaN,1
931,2023-04-27 15:00:00,330.4,333.7,329.3,331.8,39271.450,-1.423026,-1.446827,0.023801,331.8,NaN,1
904,2023-04-26 12:00:00,338.2,345.0,338.1,344.6,76243.163,2.101270,1.813964,0.287306,344.6,NaN,1
883,2023-04-25 15:00:00,330.3,332.7,329.8,332.4,28711.593,0.036703,-0.063019,0.099722,332.4,NaN,1
869,2023-04-25 01:00:00,331.1,332.4,331.1,331.6,15561.030,0.310334,0.308192,0.002142,331.6,NaN,1
863,2023-04-24 19:00:00,331.7,332.2,331.5,332.0,10528.637,0.309169,0.264865,0.044304,332.0,NaN,1
856,2023-04-24 12:00:00,331.2,332.5,331.1,332.3,21116.984,0.291411,0.269337,0.022075,332.3,NaN,1
844,2023-04-24 00:00:00,330.6,332.7,329.4,332.4,27088.801,0.429767,0.393934,0.035833,332.4,NaN,1
801,2023-04-22 05:00:00,323.4,324.1,323.1,323.7,10961.245,-0.321082,-0.396489,0.075407,323.7,NaN,1
